<a href="https://colab.research.google.com/github/yash-1708/Stock-Analyst-Rating/blob/master/scrapingmarketsmojoBIG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
import pandas.io.sql as psql
from dateutil import parser
import sqlalchemy
from datetime import datetime

In [0]:
driver = webdriver.Chrome("chromedriver.exe")
broker=[] #List to store name of the broker
company_name=[] #List to store company names 
current_price=[] #List to store rating of the product
predict_date=[]
recomended_buying=[]
target_price=[]
driver.get("https://www.marketsmojo.com/recommendation/")


In [0]:
content = driver.page_source
soup = BeautifulSoup(content,'html.parser')
Alldata=soup.select_one("div.righttexcontent2 div.right-table01 div.scr-result2")
# for row in Alldata.findAll('div',class_='middle-colum02'):
for row in Alldata.findAll('div',attrs={'class':'middle-colum02'}):
    broker.append(row.select("div.colum2-midd01 div.row div.margin-text ul.reco-price li div.text-top0 a.broker-link")[0].get_text())
    company_name.append(row.select("div.colum2-midd02 div.row div.margin-text ul.reco-price li div.text-top0 a.broker-link")[0].get_text().replace('\t','').replace('\n',''))
    current_price.append(row.select("div.colum2-midd03 div.row div.margin-text ul.reco-price li div.text-top0")[0].get_text())
    recomended_buying.append(row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.text-top0")[0].get_text())
    target_price.append(row.select("div.colum2-midd05 div.row div.margin-text ul.reco-price li div.text-top0")[0].get_text())
    temp_data=row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.colum-text-color-bott")
    temp_data=str(temp_data).split('on ')[1]
    temp_data=temp_data.split('<br/')[0]
    predict_date.append(temp_data)
    #predict_date.append(row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.colum-text-color-bott"))
     
for row in Alldata.findAll('div',attrs={'class':'middle-columactive'}):
    broker.append(row.select("div.colum2-midd01 div.row div.margin-text ul.reco-price li div.text-top0 a.broker-link")[0].get_text())
    company_name.append(row.select("div.colum2-midd02 div.row div.margin-text ul.reco-price li div.text-top0 a.broker-link")[0].get_text().replace('\t','').replace('\n',''))
    current_price.append(row.select("div.colum2-midd03 div.row div.margin-text ul.reco-price li div.text-top0")[0].get_text())
    recomended_buying.append(row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.text-top0")[0].get_text())
    target_price.append(row.select("div.colum2-midd05 div.row div.margin-text ul.reco-price li div.text-top0")[0].get_text())
    temp_data=row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.colum-text-color-bott")
    temp_data=str(temp_data).split('on ')[1]
    temp_data=temp_data.split('<br/')[0]
    predict_date.append(temp_data)

Test Cell

In [0]:
temp_data      #test

'01 Apr 16'

Classifying into different dataframes

In [0]:
df1 = pd.DataFrame({'broker':broker})  
df2 = pd.DataFrame({'company_name':company_name})
df3 = pd.DataFrame({'current_price':current_price})
df4 = pd.DataFrame({'recomended_buing':recomended_buying})
df5=pd.DataFrame({'target_price':target_price})
df6=pd.DataFrame({'predict_date':predict_date})
#dfALL = pd.DataFrame(broker,company_name,current_price,recomended_buying,target_price)


Cleaning Data

In [0]:
#splitting operation
rb=df4['recomended_buing'].str.split('Rs.').str[1] 
cp=df3['current_price'].str.split('Rs.').str[1]
tp=df5['target_price'].str.split('Rs.').str[1]
predict_date=df6['predict_date'].str.split('10').str[0]



In [0]:
dfall=pd.DataFrame({'broker':broker, 'company_name':company_name, 'current_price':cp, 'recomended_buying':rb, 'predict_date':predict_date, 'target_price':tp})
dfallclean = dfall[dfall['predict_date']!='']
dfallclean['predict_date'] =  pd.to_datetime(dfallclean['predict_date'], format='%d %b %y')

C:\Users\yash\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
dfallclean

broker          company_name current_price  \
0            Prabhudas Lilladher   Colgate Palmol. (I)       1248.25   
1                    Axis Direct       Larsen & Toubro        1310.9   
2                    Nirmal Bang        Britannia Inds          2695   
3                 Sushil Finance   Sahyadri Industries           147   
4      Geojit Financial Services        Tata Chemicals        580.95   
5      Geojit Financial Services         Maruti Suzuki       5917.05   
6                ICICIdirect.com          Sanofi India        6188.8   
7      Geojit Financial Services                   ITC         243.7   
8      Geojit Financial Services             Axis Bank           646   
9                    Axis Direct    Manappuram Finance        118.95   
10                     Edelweiss                   PVR        1483.6   
11                 CD Equisearch       India Hume Pipe         243.5   
12     Geojit Financial Services                 Emami           299   
13               ICICIdirect.com   Brigade Enterprises         197.1   
14           Prabhudas Lilladher         Cummins India        565.95   
15               ICICIdirect.com    Mahindra Logistics         332.1   
16     Geojit Financial Services   Piramal Enterprises        1926.4   
17      Cholamandalam Securities        Greaves Cotton           122   
18      Cholamandalam Securities        Hexaware Tech.         385.3   
19                   Axis Direct        Karnataka Bank          75.8   
20           Indiabulls Ventures      Balkrishna Inds.        717.25   
21                     Sharekhan                  HDFC        2047.7   
22     Geojit Financial Services   Suven Life Sciences           256   
23           Prabhudas Lilladher         Future Retail           404   
24                 CD Equisearch            Aarti Inds        1576.5   
25                     Edelweiss      Parag Milk Foods        137.65   
26               ICICIdirect.com          Abbott India          9460   
27                   Axis Direct         Gabriel India        104.05   
28                   Axis Direct         Ultratech Cem       3887.05   
29                    AnandRathi                   TCS        2231.2   
...                          ...                   ...           ...   
20080            ICICIdirect.com          Mindtree Ltd        691.15   
20081                   Religare                   TCS        2228.4   
20082            ICICIdirect.com     Blue Dart Express       2173.15   
20083           Kotak Securities              DCB Bank         197.6   
20084            ICICIdirect.com                   TCS        2228.4   
20085            ICICIdirect.com         Dredging Corp        327.35   
20086                Axis Direct                   TCS        2228.4   
20087              CD Equisearch            MT Educare         27.45   
20088    Arihant Capital Markets                   TCS        2228.4   
20089                Axis Direct   Mahindra & Mahindra         514.1   
20090             SPA Securities  J Kumar Infraproject        115.75   
20091                   Religare               Infosys        834.25   
20092                Axis Direct               Infosys        834.25   
20093                Nirmal Bang               Infosys        834.25   
20094           Kotak Securities               Infosys        834.25   
20095                   Religare                  CEAT         901.8   
20096           Kotak Securities         Hero MotoCorp        2578.5   
20097             SPA Securities    Navkar Corporation         34.55   
20098        Karvy Stock Broking           Jyothy Labs        140.55   
20099        Karvy Stock Broking              JBM Auto        149.95   
20100                  Edelweiss  Astra Microwave Prod         78.15   
20101                  Edelweiss            Tata Steel         346.4   
20102                  Edelweiss     Ipca Laboratories        957.25   
20103                Nirmal Bang      Aurobindo Pharma        605.

In [0]:
dfallclean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19327 entries, 0 to 20109
Data columns (total 6 columns):
broker               19327 non-null object
company_name         19327 non-null object
current_price        19327 non-null object
recomended_buying    19327 non-null object
predict_date         19327 non-null datetime64[ns]
target_price         19327 non-null object
dtypes: datetime64[ns](1), object(5)
memory usage: 1.0+ MB


In [0]:
database_username = 'root'
database_password = 'yash'
database_ip       = '127.0.0.1'
database_name     = 'mainschema'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))
dfallclean.to_sql(con=database_connection, name='marketsmojorecos', if_exists='append')